<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/sql_server_pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import package
!pip install censusgeocode
!pip install rets
!sudo apt-get install unixodbc-dev
!pip install pyodbc
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 744 kB/s 
     |████████████████████████████████| 62 kB 549 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unixodbc-dev is already the newest version (2.3.4-1.1ubuntu3).
uni

In [2]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,011 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 Packages [305 kB]
Hit:13 http

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key output should not be parsed (stdout is not a terminal)
100   983  100   983    0     0   5993      0 --:--:-- --:--:-- --:--:--  5957
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    831      0 --:--:-- --:--:-- --:--:--   831
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling

In [3]:
import pyodbc 
import pandas as pd 
conn = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                      SERVER = '76.184.235.171,1433',
                      DATABASE = 'RES_DB',
                      UID = 'user123',
                      PWD = 'Sunique123')

cursor = conn.cursor()
cursor.execute("SELECT * FROM [RES_DB].[dbo].[Inventory] WHERE [IsSold] = 1")
rows = cursor.fetchall()
df = pd.DataFrame()
address = []
for row in rows:
  address.append(row[1])   
cursor.close()
df['地址'] = address
df.to_csv('df1.csv')

In [4]:

def findifNull(lo_left, lo_right, la_left, la_right, research, zip):
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (PostalCode = {})'.format(lo_left, lo_right, la_left, la_right, zip))
    count = 0
    lis = []
    di = {}
    df = pd.DataFrame()
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
            di = item
    for key in di:
        lis.append(key)
    if(count >= 5):
        df = df.T
        df.columns = lis
        return df
    else:
        lo_left -= 0.008
        lo_right += 0.008
        la_left -= 0.008
        la_right += 0.008
        return findifNull(lo_left, lo_right, la_left, la_right, research, zip)

def nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column):
  result = research.search(resource = 'Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-06-30), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-)'.format(house_leftLo, house_rightLo, house_leftLa, house_rightLa))
  count = 0
  df = pd.DataFrame()
  for item in result: 
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) #?
            count += 1         
  if(count >= 5):  
    df = df.T
    df.columns = column # 
    return df
  else:
    house_leftLo -= 0.008
    house_rightLo += 0.008
    house_leftLa -= 0.008
    house_rightLa += 0.008
    return nullZiphouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, research, column)
    
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, b, time1, time2):
    basic = house_leftSqt
    count = 0
    df = pd.DataFrame()
    result = research.search(resource='Property', resource_class='Property', dmql_query='(MlsStatus = SLD), (CloseDate = 2022-01-01-2022-05-31), (YearBuilt = {}+), (YearBuilt = {}-), (Longitude = {}+), (Longitude = {}-), (Latitude = {}+), (Latitude = {}-), (LivingArea = {}+), (LivingArea = {}-), (PostalCode = {})'.format(time1, time2, house_leftLo, house_rightLo, house_leftLa, house_rightLa, house_leftSqt, house_rightSqt, zip))
    for item in result:
        if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
            df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
            count += 1
    if(count >= 5):
        df = df.T
        df.columns = column
        return df
    else:
        house_leftLa -= 0.008
        house_rightLa += 0.008
        house_leftLo -= 0.008
        house_rightLo += 0.008
        b +=1
        if(b >= 12):
          house_leftLo += 0.096
          house_rightLo -= 0.096
          house_leftLa += 0.096
          house_rightLa -= 0.096  
          return nullZiphouse(house_leftLa,house_rightLa,house_leftLo,house_rightLo,research,column)

  #Living Area
        if(a == 0):
            if(basic <= 1100):
                house_leftSqt -= 100
                house_rightSqt += 100
            elif(basic <= 2200):
                house_leftSqt -= 200
                house_rightSqt += 200
            else:
                house_leftSqt -= 300
                house_rightSqt += 300

            a += 1
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, research, column, zip, a, b, time1, time2)

    
def findfirst(result):
    count = 0
    for item in result:
        count += 1
    if(count == 0):
        return False
    else:
        return True    
    
def insertId(pulled_value, mls, rets_client):
    
    column = []
    for key in pulled_value:
        column.append(key)
    print("\nThis value exists in Website")
  # get whole row value
  #get each value
    zip = pulled_value.get('PostalCode')
    house_leftLa = float(pulled_value.get('Latitude'))
    house_rightLa = float(pulled_value.get('Latitude')) 
    house_leftLo = float(pulled_value.get('Longitude'))
    house_rightLo = float(pulled_value.get('Longitude'))
    house_leftSqt = float(pulled_value.get('LivingArea'))
    house_rightSqt = float(pulled_value.get('LivingArea'))
    time1 = int(pulled_value.get('YearBuilt'))
    time2 = int(pulled_value.get('YearBuilt'))
    if time1 < 1990:
        time1 = 0 
        time2 = 1989
    elif time1 >=1990 and time1 <2011:
        time1 = 1990
        time2 = 2010
    elif time1 >= 2011 and time1 < 2022:
        time1 = 2011
        time2 = 2021
    else:
        time1 = 2022
        time2 = 9999
    a = 0
    b = 0
    result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, b, time1, time2)        
    result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
    price = []
    for item in result1['ClosePrice']:
        item = float(item)
        price.append(item)
    result1['ClosePrice'] = price
    mean1 = result1['ClosePrice'].mean()
    top1 = list(result1['ClosePrice'])
    top1.sort(reverse = True)
    print('The mean for {} is {}'.format(mls, mean1))
    print('The top 3 for {} are {}'.format(mls, top1[0:3]))
    return result1.to_csv('{}.csv'.format(mls))

def findAdd(address, city, state, zip, rets_client):
    result = cg.address(address, city = city, state= state,zipcode = zip)
    lo_left = result[0]['coordinates']['x']
    lo_right = result[0]['coordinates']['x']
    la_left = result[0]['coordinates']['y']
    la_right = result[0]['coordinates']['y']
    result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, zip)
    result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
    price = []
    for item in result['ClosePrice']:
        item = float(item)
        price.append(item)
    result['ClosePrice'] = price
    mean = result['ClosePrice'].mean()
    top = list(result['ClosePrice'])
    top.sort(reverse = True)
    print('The mean for {}, {}, {} is {}'.format(address, state, zip, mean))
    print('The top 3 for {}, {}, {} are {}'.format(address, state, zip, top))
    return result.to_csv('{},{},{}.csv'.format(address, state, zip))
    
def get(number, finalN, code, i, rets_client, street, city, state):
    pulled_value = {}
    result = rets_client.search(resource='Property', resource_class='Property', dmql_query='(StreetNumber = {}),(StreetName = {}),(PostalCode = {})'.format(number[i],finalN[i],code[i]))
    for item in result:
        pulled_value = item
    if(len(pulled_value) != 0):
        column = []
        for key in pulled_value:
            column.append(key)
        print("\nThis value exists in Website")
  # get whole row value
  #get each value
        zip = pulled_value.get('PostalCode')
        house_leftLa = float(pulled_value.get('Latitude'))
        house_rightLa = float(pulled_value.get('Latitude')) 
        house_leftLo = float(pulled_value.get('Longitude'))
        house_rightLo = float(pulled_value.get('Longitude'))
        house_leftSqt = float(pulled_value.get('LivingArea'))
        house_rightSqt = float(pulled_value.get('LivingArea'))
        time1 = int(pulled_value.get('YearBuilt'))
        time2 = int(pulled_value.get('YearBuilt'))
        if time1 < 1990:
            time1 = 0 
            time2 = 1989
        elif time1 >=1990 and time1 <2011:
            time1 = 1990
            time2 = 2010
        elif time1 >= 2011 and time1 < 2022:
            time1 = 2011
            time2 = 2021
        else:
            time1 = 2022
            time2 = 9999
        a = 0
        b = 0
        result1 = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, rets_client, column, zip, a, b, time1, time2)
        result1 = result1[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
        price = []
        for item in result1['ClosePrice']:
            item = float(item)
            price.append(item)
        result1['ClosePrice'] = price
        mean1 = result1['ClosePrice'].mean()
        top1 = list(result1['ClosePrice'])
        top1.sort(reverse = True)
        return result1.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean1, top1[0:3]
    else:
        print('check coordinates')
        result = cg.address(street[i], city = city[i], state= state[i], zipcode = code[i])
        if(len(result) != 0):
            lo_left = result[0]['coordinates']['x']
            lo_right = result[0]['coordinates']['x']
            la_left = result[0]['coordinates']['y']
            la_right = result[0]['coordinates']['y']
            result = findifNull(lo_left, lo_right, la_left, la_right, rets_client, code[i])
            result = result[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
            price = []
            for item in result['ClosePrice']:
                item = float(item)
                price.append(item)
            result['ClosePrice'] = price
            mean = result['ClosePrice'].mean()
            top = list(result['ClosePrice'])
            top.sort(reverse = True)
            return result.to_csv('{} {} {}.csv'.format(number[i],finalN[i],code[i])), mean, top[0:3]
        else:
            print('invalid address')
            mean = 0
            top = 0
            return 0, mean, top
def judge(i):
    if i % 10 == 1:
        return 'st'
    elif i % 10 == 2:
        return 'nd'
    elif i % 10 == 3:
        return 'rd'
    else:
        return 'th'
def main():
    login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
    username = '0671181_NID'
    password = 'Rt$tg6jx'
    rets_client = Session(login_url, username, password)
    rets_client.login()
    
    data = pd.read_csv('df1.csv')        
    code = []
    name = []
    number = []
    street = []
    city = []
    state = []
    try:
      for item in data['地址']:
        a = item.split(',')[0]
        b = item.split(',')[2]
        c = item.split(',')[1]
    except IndexError:
        print("invalid address!")
    street.append(a)
    city.append(c)
    numbers = a.split()[0]
    names = a.split()[1:(len(a.split()) - 1)]
    codes = b.split()[1][0:5]
    states = b.split()[0]
    state.append(states)
    code.append(codes)
    name.append(names)
    number.append(numbers)
    finalN = []
    for item in name:
      string = ""
      for items in item:
        string += items
        string += ' '
        finalN.append(string[0:len(string) - 1])            
    for i in range(len(code)):
      result, mean, top = get(number, finalN, code, i, rets_client, street, city, state)
      print('The mean for {}, {}, {} is {}'.format(number[i],finalN[i],code[i], mean))
      print('The top 3 for {}, {}, {} are {}'.format(number[i],finalN[i],code[i], top))
    return
       
    
if __name__ == "__main__":
    print(main())

invalid address!
check coordinates
invalid address
The mean for 4368, N, 75043 is 0
The top 3 for 4368, N, 75043 are 0
None
